In [28]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import cv2

17. (5 баллов) Первая камера находится в начале координат, ее оптическая ось направлена вдоль оси `z`.     
Вторая камера сдвинута относительно первой на `1` в направлении оси `x`. Матрицы внутренних параметров обеих камер единичные.    
Сгенерировать случайный набор точек в трехмерном пространстве с `z>0`, спроектировать их на обе камеры(`cv2.projectPoints`), по проекциям найти необходимую матрицу(`cv2.findEssentialMat`).

In [29]:
k = np.eye(3)

rt2 = np.c_[R.from_rotvec([0,0,0]).as_matrix(), np.array([1, 0, 0])]

def gene_points():
    points = []
    for i in range(100):
        point = [np.random.choice(np.arange(100), 3),
                 np.random.choice(np.arange(100), 3),
                 np.random.choice(np.arange(start=1, stop=100), 3)
            	]
        points.append(point)
    return np.array(points, dtype=np.float64)

In [30]:
points = gene_points().T.reshape(-1,1,3)

In [31]:
image_points1, jac1 = cv2.projectPoints(points, np.zeros(3), np.zeros(3), np.eye(3), None)

In [32]:
image_points2, jac2 = cv2.projectPoints(points, np.zeros(3), np.array([1, 0, 0], dtype=np.float64), np.eye(3), None)

In [33]:
def fixing(array):
    array_temp = []
    for i in range(len(array)):
        array_temp.append(array[i][0])
    return np.array(array_temp, dtype=np.float64)

image_points1 = fixing(image_points1)
image_points2 = fixing(image_points2)

In [34]:

pts_1_norm = cv2.undistortPoints(np.expand_dims(image_points1, axis=1), cameraMatrix=np.eye(3), distCoeffs=None)
pts_2_norm = cv2.undistortPoints(np.expand_dims(image_points2, axis=1), cameraMatrix=np.eye(3), distCoeffs=None)

E, mask = cv2.findEssentialMat(pts_1_norm, pts_2_norm, focal=1.0, pp=(0., 0.), method=cv2.RANSAC, prob=0.999, threshold=3.0)
E

array([[ 6.49611123e-16, -5.09030349e-13,  1.60758161e-13],
       [ 5.13808182e-13,  1.45823451e-15, -7.07106781e-01],
       [-1.66045630e-13,  7.07106781e-01,  1.52492510e-15]])

18. По необходимой матрице, полученной в задаче `18`, получить позу первой камеры относительно второй.    
Для этого использовать `cv2.decomposeEssentialMat`, и из получившихся `4` решений выбрать одно, используя условие, что трехмерные точки должны быть перед камерами.

In [35]:
R1, R2, t = cv2.decomposeEssentialMat(E)

In [36]:
t = fixing(t)

Ответ

In [37]:
RT = np.c_[R1, t]
RT

array([[ 1.00000000e+00,  7.47761000e-15,  6.75687651e-15,
         1.00000000e+00],
       [-7.47761000e-15,  1.00000000e+00,  2.06225502e-15,
         2.27346372e-13],
       [-6.75687651e-15, -2.15656975e-15,  1.00000000e+00,
         7.19877623e-13]])

19. (5 баллов) Найти матрицу внутренних параметров камеры изображения    
 https://drive.google.com/file/d/1mC0PI9k4q_wJt9iAn6uosEVSJb9PcZIk/view?usp=sharing     
 предполагая, что оптическая ось проходит строго через центр изображения, а фокусное расстояние в эквиваленте `35мм` равно `0.64` дюйма.     
 Почему результат отличается от параметров, полученных методом калибрации камеры?    
 https://drive.google.com/file/d/1A4H84PLy7971Xd1ErS1bRRupWk9_TCYI/view?usp=sharing

In [38]:
img = cv2.imread('GOPR01170000.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray

array([[ 13,  14,  15, ..., 118, 121, 128],
       [ 16,  17,  18, ..., 128, 131, 134],
       [ 16,  17,  18, ..., 133, 136, 132],
       ...,
       [136, 138, 142, ...,  30,  27,  25],
       [138, 141, 144, ...,  18,  16,  15],
       [129, 132, 136, ...,  11,  11,  11]], dtype=uint8)

In [42]:
h,w = gray.shape
h,w

(540, 960)